# **<u>Applied Data Science Project </u>**
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ak-rahul/mandi-price-predictor/blob/main/Mandi-Project.ipynb)

## **1. Install required packages**

In [ ]:
!pip install gradio plotly pandas scikit-learn xgboost lightgbm seaborn -q

import gradio as gr
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed!")


✅ All packages installed!


## **2. Loading the Dataset**

In [ ]:
github_url = "https://raw.githubusercontent.com/ak-rahul/datasets/main/mandi-daily-price.csv"
df = pd.read_csv(github_url)
print("✅ Dataset loaded!")

✅ Dataset loaded!


## **3. Replacing Null Values**

In [ ]:
df.columns = df.columns.str.replace('_x0020_', '_')
print("✅ Handlled Null Values!")

✅ Handlled Null Values!


## **4. Feature Engineering**

In [ ]:
df['Price_Spread'] = df['Max_Price'] - df['Min_Price']
df['Price_Volatility_Pct'] = (df['Price_Spread'] / df['Modal_Price']) * 100


## **5. Specifying Commodities**

In [ ]:
vegetables = ['Potato', 'Tomato', 'Onion', 'Cauliflower', 'Cabbage', 'Brinjal',
              'Green Chilli', 'Carrot', 'Raddish', 'Pumpkin', 'Capsicum', 'Beans',
              'Ladies Finger', 'Bitter Gourd', 'Bottle Gourd', 'Ridge Gourd']
fruits = ['Mango', 'Apple', 'Banana', 'Orange', 'Grapes', 'Kiwi Fruit', 'Pineapple',
          'Kinnow', 'Papaya', 'Watermelon', 'Pomegranate']
spices = ['Cummin Seed(Jeera)', 'Dry Chillies', 'Chili Red', 'Turmeric', 'Coriander',
          'Black Pepper', 'Cardamom', 'Cloves', 'Ginger', 'Garlic']

df['Category'] = df['Commodity'].apply(
    lambda x: 'Vegetables' if x in vegetables else
              'Fruits' if x in fruits else
              'Spices' if x in spices else 'Others'
)

print("✅ Assigned Commodity types!")

✅ Assigned Commodity types!


## **6. Label Encoding and specifying Features**

In [ ]:
df_ml = df.copy()
encoders = {}

for col in ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Category']:
    le = LabelEncoder()
    df_ml[f'{col}_Encoded'] = le.fit_transform(df_ml[col])
    encoders[col] = le
print("✅ Completed Label Encoding!")

feature_cols = ['State_Encoded', 'District_Encoded', 'Market_Encoded',
                'Commodity_Encoded', 'Variety_Encoded', 'Grade_Encoded',
                'Category_Encoded', 'Min_Price', 'Max_Price']
print("✅ Specified Features!")

✅ Completed Label Encoding!
✅ Specified Features!


## **7. Splitting the dataset**

In [ ]:
X = df_ml[feature_cols]
y = df_ml['Modal_Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("✅ Splitted the dataset into trainng and testing!")

✅ Splitted the dataset into trainng and testing!


## **8. Training the Models**

In [ ]:
models = {}
results = []

rf_model = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
models['Random Forest'] = rf_model
rf_pred = rf_model.predict(X_test)
results.append({
    'Model': 'Random Forest',
    'RMSE': np.sqrt(mean_squared_error(y_test, rf_pred)),
    'MAE': mean_absolute_error(y_test, rf_pred),
    'R²': r2_score(y_test, rf_pred)
})

xgb_model = XGBRegressor(n_estimators=100, max_depth=7, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
models['XGBoost'] = xgb_model
xgb_pred = xgb_model.predict(X_test)
results.append({
    'Model': 'XGBoost',
    'RMSE': np.sqrt(mean_squared_error(y_test, xgb_pred)),
    'MAE': mean_absolute_error(y_test, xgb_pred),
    'R²': r2_score(y_test, xgb_pred)
})

lgbm_model = LGBMRegressor(n_estimators=100, max_depth=7, learning_rate=0.1, random_state=42, verbose=-1)
lgbm_model.fit(X_train, y_train)
models['LightGBM'] = lgbm_model
lgbm_pred = lgbm_model.predict(X_test)
results.append({
    'Model': 'LightGBM',
    'RMSE': np.sqrt(mean_squared_error(y_test, lgbm_pred)),
    'MAE': mean_absolute_error(y_test, lgbm_pred),
    'R²': r2_score(y_test, lgbm_pred)
})

results_df = pd.DataFrame(results).sort_values('R²', ascending=False)
best_model_name = results_df.iloc[0]['Model']
best_model = models[best_model_name]

print("✅ Models trained!")
print(f"🏆 Best Model: {best_model_name} (R² = {results_df.iloc[0]['R²']:.4f})")


✅ Models trained!
🏆 Best Model: Random Forest (R² = 0.9916)


## **9. Styling the Dashboard**

In [ ]:
dark_theme_css = """
@import url('https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;500;600;700;800;900&display=swap');

:root {
    --primary-color: #8b5cf6;
    --secondary-color: #a78bfa;
    --accent-color: #ec4899;
    --success-color: #10b981;
    --dark-bg: #0f172a;
    --dark-card: #1e293b;
    --dark-surface: #334155;
}

* {
    font-family: 'Poppins', -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif !important;
}

/* FIXED WIDTH - Main container */
.gradio-container {
    max-width: 1600px !important;
    width: 100% !important;
    margin: 0 auto !important;
    padding: 2rem !important;
    background: linear-gradient(135deg, #0f172a 0%, #1e293b 100%) !important;
    min-height: 100vh !important;
}

/* FIXED WIDTH - Prevent tab content from changing width */
.gr-tab-content {
    width: 100% !important;
    max-width: 1600px !important;
    overflow-x: hidden !important;
}

.gr-panel {
    width: 100% !important;
    max-width: 100% !important;
    overflow-x: hidden !important;
}

/* Hide scrollbars but keep scrolling */
.gr-tab-content, .gr-panel {
    scrollbar-width: thin !important;
}

/* Dark theme body */
body {
    background: #0f172a !important;
    color: #e2e8f0 !important;
}

/* Hide Streamlit/Gradio branding */
#MainMenu {visibility: hidden;}
footer {visibility: hidden;}
header {visibility: hidden;}

/* Title styling - Dark theme */
h1 {
    background: linear-gradient(135deg, #a78bfa 0%, #ec4899 50%, #8b5cf6 100%) !important;
    -webkit-background-clip: text !important;
    -webkit-text-fill-color: transparent !important;
    background-clip: text !important;
    font-size: 3.5rem !important;
    font-weight: 900 !important;
    text-align: center !important;
    margin: 1.5rem 0 !important;
    letter-spacing: -2px !important;
    line-height: 1.2 !important;
}

/* Section headers - Dark theme */
h2 {
    color: #f1f5f9 !important;
    font-weight: 700 !important;
    font-size: 2rem !important;
    margin: 2rem 0 1rem 0 !important;
    padding-bottom: 0.75rem !important;
    border-bottom: 3px solid transparent !important;
    border-image: linear-gradient(90deg, #8b5cf6, #ec4899) 1 !important;
}

h3 {
    color: #cbd5e1 !important;
    font-weight: 600 !important;
    font-size: 1.5rem !important;
    margin: 1.5rem 0 0.75rem 0 !important;
}

h4 {
    color: #94a3b8 !important;
    font-weight: 600 !important;
    font-size: 1.2rem !important;
}

/* Text colors for dark theme */
p, span, div {
    color: #cbd5e1 !important;
}

/* Premium button styling - Dark theme */
.gr-button {
    background: linear-gradient(135deg, #8b5cf6 0%, #a78bfa 50%, #ec4899 100%) !important;
    border: none !important;
    color: white !important;
    font-weight: 600 !important;
    font-size: 1rem !important;
    padding: 14px 32px !important;
    border-radius: 14px !important;
    box-shadow: 0 10px 40px rgba(139, 92, 246, 0.5) !important;
    transition: all 0.4s cubic-bezier(0.4, 0, 0.2, 1) !important;
    text-transform: uppercase !important;
    letter-spacing: 1px !important;
    position: relative !important;
    overflow: hidden !important;
}

.gr-button::before {
    content: '' !important;
    position: absolute !important;
    top: 0 !important;
    left: -100% !important;
    width: 100% !important;
    height: 100% !important;
    background: linear-gradient(90deg, transparent, rgba(255,255,255,0.3), transparent) !important;
    transition: left 0.5s !important;
}

.gr-button:hover::before {
    left: 100% !important;
}

.gr-button:hover {
    transform: translateY(-4px) scale(1.02) !important;
    box-shadow: 0 15px 50px rgba(139, 92, 246, 0.7) !important;
}

.gr-button:active {
    transform: translateY(-2px) scale(0.98) !important;
}

/* Cards and containers - Dark theme */
.gr-box, .gr-form, .gr-panel {
    background: linear-gradient(135deg, #1e293b 0%, #334155 100%) !important;
    border-radius: 20px !important;
    border: 1px solid rgba(139, 92, 246, 0.2) !important;
    padding: 2rem !important;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.5) !important;
    transition: all 0.3s ease !important;
    position: relative !important;
    overflow: hidden !important;
}

.gr-box::before {
    content: '' !important;
    position: absolute !important;
    top: 0 !important;
    left: 0 !important;
    width: 100% !important;
    height: 4px !important;
    background: linear-gradient(90deg, #8b5cf6, #a78bfa, #ec4899) !important;
}

.gr-box:hover {
    transform: translateY(-5px) !important;
    box-shadow: 0 15px 50px rgba(139, 92, 246, 0.3) !important;
    border-color: rgba(139, 92, 246, 0.5) !important;
}

/* Tabs styling - Dark theme */
.gr-tab-nav {
    background: #1e293b !important;
    border-radius: 16px !important;
    padding: 8px !important;
    box-shadow: 0 4px 20px rgba(0, 0, 0, 0.4) !important;
    gap: 8px !important;
    border: 1px solid rgba(139, 92, 246, 0.3) !important;
}

.gr-tab {
    background: transparent !important;
    border: none !important;
    color: #94a3b8 !important;
    font-weight: 600 !important;
    font-size: 1.1rem !important;
    padding: 16px 28px !important;
    border-radius: 12px !important;
    transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
    position: relative !important;
}

.gr-tab:hover {
    background: rgba(139, 92, 246, 0.2) !important;
    color: #a78bfa !important;
}

.gr-tab.selected {
    background: linear-gradient(135deg, #8b5cf6 0%, #a78bfa 100%) !important;
    color: white !important;
    box-shadow: 0 8px 25px rgba(139, 92, 246, 0.5) !important;
    transform: scale(1.05) !important;
}

/* Input fields - Dark theme */
.gr-input, .gr-dropdown, .gr-textbox, .gr-number {
    border-radius: 12px !important;
    border: 2px solid #334155 !important;
    background: #1e293b !important;
    color: #e2e8f0 !important;
    font-size: 1rem !important;
    padding: 12px 16px !important;
    transition: all 0.3s ease !important;
}

.gr-input:focus, .gr-dropdown:focus, .gr-textbox:focus, .gr-number:focus {
    border-color: #8b5cf6 !important;
    box-shadow: 0 0 0 4px rgba(139, 92, 246, 0.3) !important;
    outline: none !important;
    background: #334155 !important;
}

/* Dropdown options - Dark theme */
select option {
    background: #1e293b !important;
    color: #e2e8f0 !important;
}

/* Labels - Dark theme */
label {
    color: #cbd5e1 !important;
    font-weight: 600 !important;
}

/* Data table styling - Dark theme */
.gr-dataframe {
    border-radius: 16px !important;
    overflow: hidden !important;
    border: 1px solid rgba(139, 92, 246, 0.3) !important;
    box-shadow: 0 4px 25px rgba(0, 0, 0, 0.4) !important;
}

.gr-dataframe table {
    border-collapse: separate !important;
    border-spacing: 0 !important;
    background: #1e293b !important;
}

.gr-dataframe th {
    background: linear-gradient(135deg, #8b5cf6 0%, #a78bfa 100%) !important;
    color: white !important;
    font-weight: 700 !important;
    padding: 16px !important;
    text-transform: uppercase !important;
    letter-spacing: 0.5px !important;
    font-size: 0.85rem !important;
}

.gr-dataframe td {
    padding: 14px !important;
    border-bottom: 1px solid #334155 !important;
    background: #1e293b !important;
    color: #cbd5e1 !important;
}

.gr-dataframe tr:hover td {
    background: rgba(139, 92, 246, 0.1) !important;
}

/* Success/Info boxes - Dark theme */
.success-box {
    background: linear-gradient(135deg, rgba(16, 185, 129, 0.2) 0%, rgba(5, 150, 105, 0.2) 100%) !important;
    border-left: 5px solid #10b981 !important;
    border-radius: 16px !important;
    padding: 1.5rem !important;
    margin: 1.5rem 0 !important;
    box-shadow: 0 4px 20px rgba(16, 185, 129, 0.2) !important;
    color: #d1fae5 !important;
}

.success-box h3, .success-box strong {
    color: #6ee7b7 !important;
}

.info-box {
    background: linear-gradient(135deg, rgba(59, 130, 246, 0.2) 0%, rgba(37, 99, 235, 0.2) 100%) !important;
    border-left: 5px solid #3b82f6 !important;
    border-radius: 16px !important;
    padding: 1.5rem !important;
    margin: 1.5rem 0 !important;
    box-shadow: 0 4px 20px rgba(59, 130, 246, 0.2) !important;
    color: #dbeafe !important;
}

.info-box strong {
    color: #93c5fd !important;
}

.warning-box {
    background: linear-gradient(135deg, rgba(245, 158, 11, 0.2) 0%, rgba(217, 119, 6, 0.2) 100%) !important;
    border-left: 5px solid #f59e0b !important;
    border-radius: 16px !important;
    padding: 1.5rem !important;
    margin: 1.5rem 0 !important;
    box-shadow: 0 4px 20px rgba(245, 158, 11, 0.2) !important;
    color: #fef3c7 !important;
}

.warning-box strong {
    color: #fcd34d !important;
}

/* Metric/Stat cards - Dark theme */
.stat-card {
    background: linear-gradient(135deg, #1e293b 0%, #334155 100%) !important;
    border-radius: 18px !important;
    padding: 2rem !important;
    box-shadow: 0 8px 30px rgba(0, 0, 0, 0.4) !important;
    border-top: 4px solid #8b5cf6 !important;
    transition: all 0.4s cubic-bezier(0.4, 0, 0.2, 1) !important;
    position: relative !important;
    overflow: hidden !important;
    border: 1px solid rgba(139, 92, 246, 0.2) !important;
}

.stat-card::after {
    content: '' !important;
    position: absolute !important;
    top: -50% !important;
    right: -50% !important;
    width: 200% !important;
    height: 200% !important;
    background: radial-gradient(circle, rgba(139, 92, 246, 0.2) 0%, transparent 70%) !important;
    transition: all 0.6s !important;
}

.stat-card:hover {
    transform: translateY(-8px) rotate(1deg) !important;
    box-shadow: 0 15px 50px rgba(139, 92, 246, 0.4) !important;
    border-color: rgba(139, 92, 246, 0.5) !important;
}

.stat-card:hover::after {
    top: -10% !important;
    right: -10% !important;
}

/* Plot containers - Dark theme */
.plotly {
    border-radius: 16px !important;
    overflow: hidden !important;
    box-shadow: 0 4px 25px rgba(0, 0, 0, 0.4) !important;
    background: #1e293b !important;
    border: 1px solid rgba(139, 92, 246, 0.2) !important;
}

/* Markdown content - Dark theme */
.markdown-text {
    line-height: 1.8 !important;
    color: #cbd5e1 !important;
}

.markdown-text strong {
    color: #a78bfa !important;
    font-weight: 700 !important;
}

/* Loading spinner - Dark theme */
.gr-loading {
    color: #a78bfa !important;
}

/* Scrollbar styling - Dark theme */
::-webkit-scrollbar {
    width: 12px !important;
    height: 12px !important;
}

::-webkit-scrollbar-track {
    background: #1e293b !important;
    border-radius: 10px !important;
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(135deg, #8b5cf6 0%, #a78bfa 100%) !important;
    border-radius: 10px !important;
}

::-webkit-scrollbar-thumb:hover {
    background: linear-gradient(135deg, #a78bfa 0%, #8b5cf6 100%) !important;
}

/* Animations */
@keyframes fadeInUp {
    from {
        opacity: 0;
        transform: translateY(30px);
    }
    to {
        opacity: 1;
        transform: translateY(0);
    }
}

.gr-box, .gr-button, .stat-card {
    animation: fadeInUp 0.6s ease-out !important;
}

/* HTML content divs - Dark theme */
.gradio-html {
    color: #cbd5e1 !important;
}

.gradio-html div {
    color: inherit !important;
}

/* Ensure all text is visible in dark theme */
.gr-text, .gr-markdown {
    color: #cbd5e1 !important;
}

/* Row and Column spacing - maintain consistency */
.gr-row {
    width: 100% !important;
    gap: 1rem !important;
}

.gr-column {
    min-width: 0 !important;
}

/* Responsive design */
@media (max-width: 768px) {
    .gradio-container {
        padding: 1rem !important;
    }

    h1 {
        font-size: 2.5rem !important;
    }

    h2 {
        font-size: 1.5rem !important;
    }

    .gr-button {
        padding: 12px 24px !important;
        font-size: 0.9rem !important;
    }

    .stat-card {
        padding: 1.5rem !important;
    }
}
"""


## **10. Dashboard Functions**

In [ ]:
def show_overview():
    stats_html = f"""
    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 1.5rem; margin: 2rem 0;">
        <div class="stat-card">
            <div style="font-size: 3rem; margin-bottom: 0.5rem;">📈</div>
            <div style="font-size: 2.5rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{len(df):,}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase; letter-spacing: 1px; font-size: 0.9rem;">Total Records</div>
            <div style="color: #10b981; font-size: 0.85rem; margin-top: 0.5rem;">● Live Data</div>
        </div>

        <div class="stat-card">
            <div style="font-size: 3rem; margin-bottom: 0.5rem;">🌾</div>
            <div style="font-size: 2.5rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{df['Commodity'].nunique()}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase; letter-spacing: 1px; font-size: 0.9rem;">Commodities</div>
            <div style="color: #10b981; font-size: 0.85rem; margin-top: 0.5rem;">● Active Trading</div>
        </div>

        <div class="stat-card">
            <div style="font-size: 3rem; margin-bottom: 0.5rem;">🏪</div>
            <div style="font-size: 2.5rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{df['Market'].nunique()}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase; letter-spacing: 1px; font-size: 0.9rem;">Markets</div>
            <div style="color: #10b981; font-size: 0.85rem; margin-top: 0.5rem;">● Nationwide</div>
        </div>

        <div class="stat-card">
            <div style="font-size: 3rem; margin-bottom: 0.5rem;">🗺️</div>
            <div style="font-size: 2.5rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{df['State'].nunique()}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase; letter-spacing: 1px; font-size: 0.9rem;">States Covered</div>
            <div style="color: #10b981; font-size: 0.85rem; margin-top: 0.5rem;">● Pan India</div>
        </div>
    </div>

    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 1.5rem; margin: 2rem 0;">
        <div class="info-box">
            <div style="font-weight: 700; font-size: 1.1rem; margin-bottom: 0.5rem; color: #93c5fd;">💰 Average Price</div>
            <div style="font-size: 2rem; font-weight: 800; color: #60a5fa;">₹{df['Modal_Price'].mean():.2f}</div>
        </div>

        <div class="success-box">
            <div style="font-weight: 700; font-size: 1.1rem; margin-bottom: 0.5rem; color: #6ee7b7;">📊 Median Price</div>
            <div style="font-size: 2rem; font-weight: 800; color: #34d399;">₹{df['Modal_Price'].median():.2f}</div>
        </div>

        <div class="warning-box">
            <div style="font-weight: 700; font-size: 1.1rem; margin-bottom: 0.5rem; color: #fcd34d;">🔥 Maximum Price</div>
            <div style="font-size: 2rem; font-weight: 800; color: #fbbf24;">₹{df['Modal_Price'].max():.2f}</div>
        </div>
    </div>
    """

    sample_df = df[['State', 'Market', 'Commodity', 'Variety', 'Min_Price', 'Max_Price', 'Modal_Price']].head(20)
    return stats_html, sample_df

def create_viz(viz_type):
    plot_template = {
        'layout': {
            'plot_bgcolor': 'rgba(15, 23, 42, 0.8)',
            'paper_bgcolor': 'rgba(30, 41, 59, 0.8)',
            'font': {'family': 'Poppins', 'size': 13, 'color': '#cbd5e1'},
            'title': {'font': {'size': 20, 'color': '#f1f5f9'}},
            'xaxis': {'gridcolor': 'rgba(203, 213, 225, 0.1)', 'color': '#cbd5e1'},
            'yaxis': {'gridcolor': 'rgba(203, 213, 225, 0.1)', 'color': '#cbd5e1'}
        }
    }

    if viz_type == "Top 20 Commodities by Price":
        top_commodities = df.groupby('Commodity')['Modal_Price'].mean().sort_values(ascending=False).head(20)
        fig = px.bar(x=top_commodities.values, y=top_commodities.index, orientation='h',
                     title='<b>Top 20 Commodities by Average Price</b>',
                     labels={'x': 'Average Price (₹)', 'y': 'Commodity'},
                     color=top_commodities.values, color_continuous_scale='viridis')
        fig.update_layout(height=700, showlegend=False, **plot_template['layout'])

    elif viz_type == "State-wise Price Distribution":
        state_prices = df.groupby('State')['Modal_Price'].mean().sort_values(ascending=False).head(15)
        fig = px.bar(x=state_prices.index, y=state_prices.values,
                     title='<b>Top 15 States by Average Price</b>',
                     labels={'x': 'State', 'y': 'Average Price (₹)'},
                     color=state_prices.values, color_continuous_scale='plasma')
        fig.update_layout(xaxis_tickangle=-45, height=600, **plot_template['layout'])

    elif viz_type == "Price Distribution by Category":
        fig = px.box(df, x='Category', y='Modal_Price',
                     title='<b>Price Distribution by Commodity Category</b>',
                     color='Category', color_discrete_sequence=px.colors.qualitative.Vivid)
        fig.update_layout(height=600, **plot_template['layout'])

    elif viz_type == "Price Volatility Analysis":
        volatility = df.groupby('Commodity').agg({'Price_Volatility_Pct': 'mean', 'Modal_Price': 'count'})
        volatility = volatility[volatility['Modal_Price'] >= 5].sort_values('Price_Volatility_Pct', ascending=False).head(15)
        fig = px.bar(x=volatility.index, y=volatility['Price_Volatility_Pct'],
                     title='<b>Top 15 Most Volatile Commodities</b>',
                     color=volatility['Price_Volatility_Pct'], color_continuous_scale='Reds')
        fig.update_layout(xaxis_tickangle=-45, height=600, **plot_template['layout'])

    elif viz_type == "Price Spread vs Modal Price":
        fig = px.scatter(df.sample(500), x='Modal_Price', y='Price_Spread', color='Category',
                         hover_data=['Commodity', 'State'], title='<b>Price Spread vs Modal Price</b>',
                         opacity=0.7, color_discrete_sequence=px.colors.qualitative.Vivid)
        fig.update_layout(height=600, **plot_template['layout'])

    else:  # Top 10 Markets
        market_counts = df['Market'].value_counts().head(10)
        fig = px.bar(x=market_counts.index, y=market_counts.values,
                     title='<b>Top 10 Markets by Number of Items</b>',
                     color=market_counts.values, color_continuous_scale='Blues')
        fig.update_layout(xaxis_tickangle=-45, height=600, **plot_template['layout'])

    return fig

def predict_price(state, district, market, commodity, variety, grade, min_price, max_price):
    if not all([state, district, market, commodity, variety, grade]):
        return "<div class='warning-box'><strong>⚠️ Warning:</strong> Please fill in all fields to make a prediction</div>", None

    try:
        category = 'Vegetables' if commodity in vegetables else \
                  'Fruits' if commodity in fruits else \
                  'Spices' if commodity in spices else 'Others'

        input_data = pd.DataFrame({
            'State': [state], 'District': [district], 'Market': [market],
            'Commodity': [commodity], 'Variety': [variety], 'Grade': [grade],
            'Category': [category]
        })

        for col in ['State', 'District', 'Market', 'Commodity', 'Variety', 'Grade', 'Category']:
            try:
                input_data[f'{col}_Encoded'] = encoders[col].transform(input_data[col])
            except:
                return f"<div class='warning-box'><strong>❌ Error:</strong> '{input_data[col].iloc[0]}' not found in training data for {col}</div>", None

        input_data['Min_Price'] = float(min_price)
        input_data['Max_Price'] = float(max_price)
        X_input = input_data[feature_cols]

        predictions = {}
        for name, model in models.items():
            pred = model.predict(X_input)[0]
            predictions[name] = pred

        result = f"""
<div class="success-box">
<h3 style="margin-top: 0; color: #6ee7b7;">🎯 Price Prediction Results</h3>

<div style="background: rgba(30, 41, 59, 0.8); padding: 1.5rem; border-radius: 12px; margin: 1rem 0; border: 1px solid rgba(139, 92, 246, 0.3);">
    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 1rem;">
        <div>
            <strong style="color: #a78bfa;">📍 Location:</strong><br>
            <span style="color: #cbd5e1;">{state} → {district} → {market}</span>
        </div>
        <div>
            <strong style="color: #a78bfa;">🌾 Commodity:</strong><br>
            <span style="color: #cbd5e1;">{commodity} ({variety}) - {grade}</span>
        </div>
    </div>
</div>

<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 1rem; margin: 1.5rem 0;">
    <div style="background: linear-gradient(135deg, rgba(59, 130, 246, 0.3) 0%, rgba(37, 99, 235, 0.3) 100%); padding: 1.5rem; border-radius: 12px; text-align: center; border: 1px solid rgba(59, 130, 246, 0.4);">
        <div style="font-size: 1.1rem; font-weight: 600; color: #93c5fd; margin-bottom: 0.5rem;">🌳 Random Forest</div>
        <div style="font-size: 2rem; font-weight: 800; color: #60a5fa;">₹{predictions['Random Forest']:.2f}</div>
    </div>
    <div style="background: linear-gradient(135deg, rgba(245, 158, 11, 0.3) 0%, rgba(217, 119, 6, 0.3) 100%); padding: 1.5rem; border-radius: 12px; text-align: center; border: 1px solid rgba(245, 158, 11, 0.4);">
        <div style="font-size: 1.1rem; font-weight: 600; color: #fcd34d; margin-bottom: 0.5rem;">⚡ XGBoost</div>
        <div style="font-size: 2rem; font-weight: 800; color: #fbbf24;">₹{predictions['XGBoost']:.2f}</div>
    </div>
    <div style="background: linear-gradient(135deg, rgba(16, 185, 129, 0.3) 0%, rgba(5, 150, 105, 0.3) 100%); padding: 1.5rem; border-radius: 12px; text-align: center; border: 1px solid rgba(16, 185, 129, 0.4);">
        <div style="font-size: 1.1rem; font-weight: 600; color: #6ee7b7; margin-bottom: 0.5rem;">💚 LightGBM</div>
        <div style="font-size: 2rem; font-weight: 800; color: #34d399;">₹{predictions['LightGBM']:.2f}</div>
    </div>
</div>
</div>

<div class="info-box">
<strong style="color: #93c5fd;">💡 Recommendation:</strong> Based on the <strong style="color: #a78bfa;">{best_model_name}</strong> model (R² = {results_df.iloc[0]['R²']:.4f}),
the predicted modal price is <strong style="font-size: 1.3rem; color: #a78bfa;">₹{predictions[best_model_name]:.2f}</strong>
</div>
        """

        pred_df = pd.DataFrame({'Model': list(predictions.keys()), 'Predicted Price': list(predictions.values())})
        fig = px.bar(pred_df, x='Model', y='Predicted Price', title='<b>Price Predictions Comparison</b>',
                     color='Predicted Price', color_continuous_scale='Viridis', text='Predicted Price')
        fig.update_traces(texttemplate='₹%{text:.2f}', textposition='outside')
        fig.add_hline(y=min_price, line_dash="dash", line_color="#ef4444", annotation_text="Min Price")
        fig.add_hline(y=max_price, line_dash="dash", line_color="#22c55e", annotation_text="Max Price")
        fig.update_layout(
            height=500, showlegend=False,
            plot_bgcolor='rgba(15, 23, 42, 0.8)',
            paper_bgcolor='rgba(30, 41, 59, 0.8)',
            font={'family': 'Poppins', 'size': 13, 'color': '#cbd5e1'},
            title={'font': {'size': 20, 'color': '#f1f5f9'}},
            xaxis={'color': '#cbd5e1'},
            yaxis={'gridcolor': 'rgba(203, 213, 225, 0.1)', 'color': '#cbd5e1'}
        )

        return result, fig
    except Exception as e:
        return f"<div class='warning-box'><strong>❌ Error:</strong> {str(e)}</div>", None

def show_models():
    results_display = results_df.copy()
    results_display['RMSE'] = results_display['RMSE'].apply(lambda x: f'₹{x:.2f}')
    results_display['MAE'] = results_display['MAE'].apply(lambda x: f'₹{x:.2f}')
    results_display['R²'] = results_display['R²'].apply(lambda x: f'{x:.4f}')

    model_html = f"""
    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 1.5rem; margin: 2rem 0;">
        <div class="stat-card">
            <div style="font-size: 2.5rem; margin-bottom: 0.5rem;">🤖</div>
            <div style="font-size: 2rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">3</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase;">Models Trained</div>
        </div>
        <div class="stat-card">
            <div style="font-size: 2.5rem; margin-bottom: 0.5rem;">📊</div>
            <div style="font-size: 2rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{len(X_train):,}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase;">Training Samples</div>
        </div>
        <div class="stat-card">
            <div style="font-size: 2.5rem; margin-bottom: 0.5rem;">🔢</div>
            <div style="font-size: 2rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{len(feature_cols)}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase;">Features Used</div>
        </div>
        <div class="stat-card">
            <div style="font-size: 2.5rem; margin-bottom: 0.5rem;">🏆</div>
            <div style="font-size: 1.5rem; font-weight: 800; background: linear-gradient(135deg, #a78bfa 0%, #ec4899 100%); -webkit-background-clip: text; -webkit-text-fill-color: transparent;">{best_model_name}</div>
            <div style="color: #94a3b8; font-weight: 600; text-transform: uppercase;">Best Model</div>
        </div>
    </div>
    """

    fig = make_subplots(rows=1, cols=3, subplot_titles=('RMSE', 'MAE', 'R² Score'))
    fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['RMSE'], marker_color='#ef4444'), row=1, col=1)
    fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['MAE'], marker_color='#3b82f6'), row=1, col=2)
    fig.add_trace(go.Bar(x=results_df['Model'], y=results_df['R²'], marker_color='#22c55e'), row=1, col=3)
    fig.update_layout(
        height=500, showlegend=False, title_text="<b>Model Metrics Comparison</b>",
        plot_bgcolor='rgba(15, 23, 42, 0.8)',
        paper_bgcolor='rgba(30, 41, 59, 0.8)',
        font={'family': 'Poppins', 'size': 13, 'color': '#cbd5e1'},
        title={'font': {'size': 20, 'color': '#f1f5f9'}}
    )
    for i in range(1, 4):
        fig.update_xaxes(color='#cbd5e1', row=1, col=i)
        fig.update_yaxes(gridcolor='rgba(203, 213, 225, 0.1)', color='#cbd5e1', row=1, col=i)

    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    fig2 = px.bar(feature_importance, x='Importance', y='Feature', orientation='h',
                  title=f'<b>Feature Importance - {best_model_name}</b>',
                  color='Importance', color_continuous_scale='Viridis')
    fig2.update_layout(
        height=500,
        plot_bgcolor='rgba(15, 23, 42, 0.8)',
        paper_bgcolor='rgba(30, 41, 59, 0.8)',
        font={'family': 'Poppins', 'size': 13, 'color': '#cbd5e1'},
        title={'font': {'size': 20, 'color': '#f1f5f9'}},
        xaxis={'gridcolor': 'rgba(203, 213, 225, 0.1)', 'color': '#cbd5e1'},
        yaxis={'color': '#cbd5e1'}
    )

    return model_html, results_display, fig, fig2

print("✅ Dashboard functions created!")

✅ Dashboard functions created!


## **11. Build Interactive Dashboard**


In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), css=dark_theme_css, title="🌾 Mandi Analytics") as dashboard:

    gr.Markdown("""
    # 🌾 Agricultural Commodity Price Analytics
    ## ML-Powered Market Intelligence Platform for Indian Agriculture

    <div style="text-align: center; padding: 1.5rem; background: linear-gradient(135deg, rgba(139, 92, 246, 0.2) 0%, rgba(236, 72, 153, 0.2) 100%); border-radius: 16px; margin: 1.5rem 0; border: 1px solid rgba(139, 92, 246, 0.3);">
        <strong style="font-size: 1.2rem; color: #a78bfa;">Real-time commodity price analysis • Advanced ML predictions • Comprehensive market insights</strong>
    </div>
    """)

    with gr.Tabs():
        with gr.Tab("📊 Overview"):
            gr.Markdown("## 📊 Dataset Overview & Statistics")
            overview_btn = gr.Button("🔄 Load Overview Dashboard", variant="primary", size="lg")
            overview_stats = gr.HTML()
            gr.Markdown("### 📋 Sample Data Preview")
            overview_table = gr.Dataframe(wrap=True)
            overview_btn.click(fn=show_overview, outputs=[overview_stats, overview_table])

        with gr.Tab("📈 Visualizations"):
            gr.Markdown("## 📈 Interactive Data Visualizations")
            with gr.Row():
                viz_dropdown = gr.Dropdown(
                    choices=["Top 20 Commodities by Price", "State-wise Price Distribution",
                            "Price Distribution by Category", "Price Volatility Analysis",
                            "Price Spread vs Modal Price", "Top 10 Markets by Items"],
                    value="Top 20 Commodities by Price",
                    label="📊 Select Visualization Type",
                    scale=3
                )
                viz_btn = gr.Button("📊 Generate", variant="primary", size="lg", scale=1)
            viz_plot = gr.Plot()
            viz_btn.click(fn=create_viz, inputs=viz_dropdown, outputs=viz_plot)

        with gr.Tab("🎯 Predictions"):
            gr.Markdown("## 🎯 ML Price Prediction Engine")
            gr.Markdown("### Enter commodity and market details to predict modal prices")

            with gr.Row():
                with gr.Column():
                    gr.Markdown("#### 📍 Location Details")
                    state_input = gr.Dropdown(choices=sorted(df['State'].unique()), label="State")
                    district_input = gr.Dropdown(choices=sorted(df['District'].unique()), label="District")
                    market_input = gr.Dropdown(choices=sorted(df['Market'].unique()), label="Market")
                with gr.Column():
                    gr.Markdown("#### 🌾 Commodity Details")
                    commodity_input = gr.Dropdown(choices=sorted(df['Commodity'].unique()), label="Commodity")
                    variety_input = gr.Dropdown(choices=sorted(df['Variety'].unique()), label="Variety")
                    grade_input = gr.Dropdown(choices=sorted(df['Grade'].unique()), label="Grade")

            with gr.Row():
                min_price_input = gr.Number(label="💰 Minimum Price (₹)", value=1000)
                max_price_input = gr.Number(label="💰 Maximum Price (₹)", value=1500)

            predict_btn = gr.Button("🔮 Predict Modal Price", variant="primary", size="lg")
            prediction_output = gr.HTML()
            prediction_plot = gr.Plot()

            predict_btn.click(
                fn=predict_price,
                inputs=[state_input, district_input, market_input, commodity_input,
                       variety_input, grade_input, min_price_input, max_price_input],
                outputs=[prediction_output, prediction_plot]
            )

        with gr.Tab("🤖 Models"):
            gr.Markdown("## 🤖 Machine Learning Model Performance")
            model_btn = gr.Button("📊 Show Analysis", variant="primary", size="lg")
            model_stats = gr.HTML()
            gr.Markdown("### 📊 Performance Metrics")
            model_table = gr.Dataframe()
            model_plot1 = gr.Plot()
            model_plot2 = gr.Plot()
            model_btn.click(fn=show_models, outputs=[model_stats, model_table, model_plot1, model_plot2])

    gr.Markdown("""
    ---
    <div style="text-align: center; padding: 2rem; background: linear-gradient(135deg, #1e293b 0%, #334155 100%); border-radius: 16px; margin-top: 2rem; border: 1px solid rgba(139, 92, 246, 0.3);">
        <div style="font-size: 1.1rem; font-weight: 600; margin-bottom: 1rem; color: #a78bfa;">🚀 Built with Premium Technology Stack</div>
        <div style="color: #94a3b8;">
            <strong style="color: #cbd5e1;">ML Framework:</strong> XGBoost • LightGBM • Random Forest • Scikit-learn<br>
            <strong style="color: #cbd5e1;">Visualization:</strong> Plotly • Gradio<br>
            <strong style="color: #cbd5e1;">Data Source:</strong> AGMARKNET Portal - Government of India<br>
            <strong style="color: #cbd5e1;">Project Type:</strong> Applied Data Science
        </div>
    </div>
    """)

print("✅ Dashboard built successfully!")

✅ Dashboard built successfully!


## **12. Launch Dashboard**


In [ ]:
print("\n" + "="*70)
print("🚀 LAUNCHING DARK THEME DASHBOARD...")
print("="*70)

dashboard.launch(
    share=True,
    debug=False,
    inline=False,
    inbrowser=True
)

print("\n✅ Dashboard is LIVE!")


🚀 LAUNCHING DARK THEME DASHBOARD...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32236e82bec25440a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

✅ Dashboard is LIVE!
